In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2
import keras
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
age_data = pd.read_csv('/Users/blakemyers/Desktop/age_gender.csv')
age_data.head(3)

,age,ethnicity,gender,img_name,pixels
0,1,2,0,20161219203650636.jpg.chip.jpg,129 128 128 126 127 130 133 135 139 142 145 14...
1,1,2,0,20161219222752047.jpg.chip.jpg,164 74 111 168 169 171 175 182 184 188 193 199...
2,1,2,0,20161219222832191.jpg.chip.jpg,67 70 71 70 69 67 70 79 90 103 116 132 145 155...


In [3]:
def label_age (row):
    if row['age'] > 0 and row['age'] < 6:
        return 0
    elif row['age'] > 5 and row['age'] < 13:
        return 1
    elif row['age'] > 12 and row['age'] < 21:
        return 2
    elif row['age'] > 20 and row['age'] < 36:
        return 3
    elif row['age'] > 35 and row['age'] < 51:
        return 4
    elif row['age'] > 50 and row['age'] < 71:
        return 5
    else:
        return 6

In [4]:
age_data['AgeGroup'] = age_data.apply (lambda row: label_age(row), axis=1)

In [5]:
age_data.head(3)

,age,ethnicity,gender,img_name,pixels,AgeGroup
0,1,2,0,20161219203650636.jpg.chip.jpg,129 128 128 126 127 130 133 135 139 142 145 14...,0
1,1,2,0,20161219222752047.jpg.chip.jpg,164 74 111 168 169 171 175 182 184 188 193 199...,0
2,1,2,0,20161219222832191.jpg.chip.jpg,67 70 71 70 69 67 70 79 90 103 116 132 145 155...,0


In [6]:
n = len(pd.unique(age_data['age']))

In [7]:
n

104

In [8]:
age_data["age"].max()

116

In [9]:
X = []
y = []
for index, row in age_data.iterrows():
    k = row['pixels'].split(" ")
    X.append(np.array(k))
    y.append(row['AgeGroup'])
        
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [10]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

y_train= np_utils.to_categorical(y_train, num_classes = 7)
y_test = np_utils.to_categorical(y_test, num_classes = 7)

In [11]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(48,48,1)))
model.add(Convolution2D(64, (3, 3), activation="relu"))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation="relu"))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation="relu"))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation="relu"))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation="relu"))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation="relu"))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation="relu"))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation="relu"))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation="relu"))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation="relu"))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation="relu"))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation="relu"))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation="relu"))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [12]:
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(X_train,y_train,batch_size=32,epochs=30,verbose=1,validation_data=(X_test, y_test))

Train on 18964 samples, validate on 4741 samples
Epoch 1/30
18964/18964 [==============================] - 1478s 78ms/step - loss: 1.6406 - accuracy: 0.4300 - val_loss: 1.4987 - val_accuracy: 0.4666
Epoch 2/30
18964/18964 [==============================] - 1443s 76ms/step - loss: 1.4008 - accuracy: 0.4953 - val_loss: 1.2212 - val_accuracy: 0.5636
Epoch 3/30
18964/18964 [==============================] - 1440s 76ms/step - loss: 1.1847 - accuracy: 0.5584 - val_loss: 1.0734 - val_accuracy: 0.5965
Epoch 4/30
18964/18964 [==============================] - 1437s 76ms/step - loss: 1.0537 - accuracy: 0.5940 - val_loss: 0.9934 - val_accuracy: 0.6106
Epoch 5/30
18964/18964 [==============================] - 1438s 76ms/step - loss: 0.9644 - accuracy: 0.6219 - val_loss: 0.9891 - val_accuracy: 0.6176
Epoch 6/30
18964/18964 [==============================] - 1438s 76ms/step - loss: 0.9007 - accuracy: 0.6439 - val_loss: 0.9419 - val_accuracy: 0.6222
Epoch 7/30
18964/18964 [===========================

In [14]:
local_path = '/Users/blakemyers/Desktop/Jupyter/DeepLearning2021/TrainedModelSp21v3b'
model.save(local_path)

In [15]:
y_pred=model.predict_classes(X_test)

In [16]:
cm = confusion_matrix(y_test.argmax(axis=1), y_pred)

In [17]:
cm

array([[ 431,   32,    0,   10,    1,    2,    0],
       [  25,  110,   37,   21,    1,    1,    0],
       [   2,   27,  101,  171,    3,    3,    0],
       [   4,    2,   66, 1751,  205,   41,    3],
       [   2,    0,    5,  356,  260,  138,    3],
       [   0,    1,    2,   93,  117,  406,   74],
       [   1,    0,    0,   11,    7,   89,  126]])

In [18]:
import matplotlib.pyplot as plt

In [19]:
fig,ax= plt.subplots(figsize=(7,7))
sns.heatmap(cm / (cm.astype(np.float).sum(axis=1) + 1e-9), annot=False, ax=ax)
ax.set_xlabel('Predicted', size=20);
ax.set_ylabel('True', size=20);
ax.set_title('Confusion Matrix', size=20);
ax.xaxis.set_ticklabels(['1-5','6-12','13-20','21-35', '36-50', '51-70','71-116'], size=9); \
ax.yaxis.set_ticklabels(['1-5','6-12','13-20','21-35', '36-50', '51-70','71-116'], size=9);